# T2M (2-meter Temperature) GRIB2 File - Exploratory Data Analysis

This notebook is designed to explore and understand the structure of GRIB2 files containing 2-meter temperature data from ICON-D2 model for Germany.

**Goal**: Unfold and understand every aspect of the GRIB2 file in a structured way, similar to how we would explore a pandas DataFrame.

**File to analyze**: `../data/icon-d2_germany_regular-lat-lon_single-level_2025110700_000_2d_t_2m.grib2`

We'll proceed step-by-step to:
1. Load the file
2. Inspect how many parameters are present
3. Understand each parameter in detail
4. Explore relationships between parameters
5. Visualize the data structure and content

## 1. Import Required Libraries

Let's start by importing the necessary libraries:
- **xarray**: For working with multi-dimensional labeled arrays (think of it as pandas for N-dimensional data)
- **cfgrib**: Backend engine for xarray to read GRIB files
- **pandas**: For familiar tabular data operations when we convert data
- **numpy**: For numerical operations
- **matplotlib**: For visualization

In [6]:
import xarray as xr
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# For better display
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

print("Libraries imported successfully!")

Libraries imported successfully!


## 2. Load the GRIB2 File

We'll load the GRIB2 file using xarray with the cfgrib engine. This will give us a Dataset object, which is similar to having multiple related DataFrames with shared indices.

In [7]:
import bz2

# Load the GRIB2 file
file_path = '../data/icon-d2_germany_regular-lat-lon_single-level_2025110700_000_2d_t_2m.grib2.bz2'
decompressed_path = '../data/icon-d2_germany_regular-lat-lon_single-level_2025110700_000_2d_t_2m.grib2'  # Temporary decompressed file

# Decompress the file
with bz2.open(file_path, 'rb') as f_in, open(decompressed_path, 'wb') as f_out:
    f_out.write(f_in.read())

# Open the decompressed file using xarray
ds = xr.open_dataset(decompressed_path, engine='cfgrib')

print(f"File loaded successfully!")
print(f"Dataset type: {type(ds)}")

# Optional: Clean up the decompressed file after use
import os
os.remove(decompressed_path)

Ignoring index file '../data/icon-d2_germany_regular-lat-lon_single-level_2025110700_000_2d_t_2m.grib2.5b7b6.idx' older than GRIB file


File loaded successfully!
Dataset type: <class 'xarray.core.dataset.Dataset'>


## 3. First Look at the Dataset

Let's take our first look at what's inside the dataset. Think of this as similar to running `df.info()` or `df.head()` in pandas.

In [ ]:
# Display the dataset structure
# This shows all parameters (variables), coordinates (dimensions), and attributes (metadata)
ds

<xarray.Dataset> Size: 4MB
Dimensions:            (latitude: 746, longitude: 1215)
Coordinates:
  * latitude           (latitude) float64 6kB 43.18 43.2 43.22 ... 58.06 58.08
  * longitude          (longitude) float64 10kB -3.94 -3.92 -3.9 ... 20.32 20.34
    time               datetime64[ns] 8B ...
    step               timedelta64[ns] 8B ...
    heightAboveGround  float64 8B ...
    valid_time         datetime64[ns] 8B ...
Data variables:
    t2m                (latitude, longitude) float32 4MB ...
Attributes:
    GRIB_edition:            2
    GRIB_centre:             edzw
    GRIB_centreDescription:  Offenbach
    GRIB_subCentre:          255
    Conventions:             CF-1.7
    institution:             Offenbach
    history:                 2025-11-07T06:28 GRIB to CDM+CF via cfgrib-0.9.1...